In [1]:
#Converting the idx format data into csv
def convert(imgf, labelf, outf, n):
    f = open(imgf, "rb")
    o = open(outf, "w")
    l = open(labelf, "rb")

    f.read(16)
    l.read(8)
    images = []

    for i in range(n):
        image = [ord(l.read(1))]
        for j in range(28*28):
            image.append(ord(f.read(1)))
        images.append(image)

    for image in images:
        o.write(",".join(str(pix) for pix in image)+"\n")
    f.close()
    o.close()
    l.close()

convert("train-images.idx3-ubyte", "train-labels.idx1-ubyte",
        "train.csv", 60000)

In [25]:
#Check tensorflow and Keras version
import tensorflow as tf
print ("TensorFlow version: " + tf.__version__)
#import tensorflow.keras
print("Keras version: "  + tf.keras.__version__)

TensorFlow version: 2.1.0-rc0
Keras version: 2.2.4-tf


In [ ]:
#Import packages
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense, Activation, Dropout

import pandas as pd
import numpy as np

In [ ]:
# Read data
train = pd.read_csv('./train.csv')
labels = train.iloc[:,0].values.astype('int32')
X_train = (train.iloc[:,1:].values).astype('float32')
X_test = (pd.read_csv('./test.csv').values).astype('float32')

# convert list of labels to binary class matrix
y_train = to_categorical(labels)

In [ ]:
# pre-processing: divide by max and substract mean
scale = np.max(X_train)
X_train /= scale
X_test /= scale

mean = np.std(X_train)
X_train -= mean
X_test -= mean

In [12]:
input_dim = X_train.shape[1]
nb_classes = y_train.shape[1]

# MLP
model = Sequential()
model.add(Dense(512, input_dim=input_dim))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))

# we'll use categorical cross entropy for the loss, and RMSprop as the optimizer
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

#Training and prediction for test data
print("Training...")
model.fit(X_train, y_train, epochs=28, batch_size=128, validation_split=0.1, verbose=1)

print("Generating test predictions...")
preds = model.predict_classes(X_test, verbose=0)

Training...
Train on 53999 samples, validate on 6000 samples
Epoch 1/28
53999/53999 [==============================] - 10s 177us/sample - loss: 0.3244 - val_loss: 0.1124
Epoch 2/28
53999/53999 [==============================] - 9s 160us/sample - loss: 0.1438


KeyboardInterrupt: 

In [ ]:
#Write prediction to file
def write_preds(preds, fname):
    pd.DataFrame({"ImageId": list(range(1,len(preds)+1)), "Label": preds}).to_csv(fname, index=False, header=True)

write_preds(preds, "result.csv")